In [1]:
#pip install LibRecommender
# use anaconda kernel
import numpy as np
import pandas as pd
# import torch
# import torchvision
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF  # pure data, 
from libreco.evaluation import evaluate

d:\anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"




Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import random
import warnings
import zipfile
from pathlib import Path

import pandas as pd
import tensorflow as tf
import tqdm
warnings.filterwarnings("ignore")


def load_ml_1m():
    # download and extract zip file
    tf.keras.utils.get_file(
        "ml-1m.zip",
        "http://files.grouplens.org/datasets/movielens/ml-1m.zip",
        cache_dir=".",
        cache_subdir=".",
        extract=True,
    )
    # read and merge data into same table
    cur_path = Path(".").absolute()
    ratings = pd.read_csv(
        cur_path / "ml-1m" / "ratings.dat",
        sep="::",
        usecols=[0, 1, 2, 3],
        names=["user", "item", "rating", "time"],
    )
    users = pd.read_csv(
        cur_path / "ml-1m" / "users.dat",
        sep="::",
        usecols=[0, 1, 2, 3],
        names=["user", "sex", "age", "occupation"],
    )
    items = pd.read_csv(
        cur_path / "ml-1m" / "movies.dat",
        sep="::",
        usecols=[0, 1, 2],
        names=["item", "name", "genre"],
        encoding="iso-8859-1",
    )
    items[["genre1", "genre2", "genre3"]] = (
        items["genre"].str.split(r"|", expand=True).fillna("missing").iloc[:, :3]
    )
    items.drop("genre", axis=1, inplace=True)
    data = ratings.merge(users, on="user").merge(items, on="item")
    data.rename(columns={"rating": "label"}, inplace=True)
    # random shuffle data
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    return data



data = load_ml_1m()
data.shape
data.iloc[random.choices(range(len(data)), k=10)]  # randomly select 10 rows

,user,item,label,time,sex,age,occupation,name,genre1,genre2,genre3
351723,660,3504,3,976740059,M,45,16,Network (1976),Comedy,Drama,missing
846985,3421,2504,4,967412033,M,45,7,200 Cigarettes (1999),Comedy,Drama,missing
277141,1599,3702,3,974756938,M,25,0,Mad Max (1979),Action,Sci-Fi,missing
699266,1907,1927,5,974691674,M,35,17,All Quiet on the Western Front (1930),War,missing,missing
643513,1255,1958,2,974823432,M,25,17,Terms of Endearment (1983),Comedy,Drama,missing
52114,5485,260,5,959832987,M,25,4,Star Wars: Episode IV - A New Hope (1977),Action,Adventure,Fantasy
154883,3053,2539,3,970169878,F,25,3,Analyze This (1999),Comedy,missing,missing
927451,356,3499,5,976318285,M,56,20,Misery (1990),Horror,missing,missing
779588,5204,260,5,961631626,M,50,13,Star Wars: Episode IV - A New Hope (1977),Action,Adventure,Fantasy
241782,4430,781,3,965102575,F,25,0,Stealing Beauty (1996),Drama,missing,missing


In [3]:
# Splitting the DataFrame based on the 'sex' column
data_male = data[data['sex'] == 'M']
data_female = data[data['sex'] == 'F']

data_male
data_female

,user,item,label,time,sex,age,occupation,name,genre1,genre2,genre3
0,5755,184,3,958280246,F,35,2,Nadja (1994),Drama,missing,missing
11,4268,2098,4,965306369,F,45,20,Son of Flubber (1963),Children's,Comedy,missing
15,3770,590,5,966050693,F,35,3,Dances with Wolves (1990),Adventure,Drama,Western
16,1602,2028,4,974733468,F,25,7,Saving Private Ryan (1998),Action,Drama,War
19,4517,1381,1,991034706,F,18,1,Grease 2 (1982),Comedy,Musical,Romance
...,...,...,...,...,...,...,...,...,...,...,...
1000198,3592,560,4,966646657,F,56,2,"Beans of Egypt, Maine, The (1994)",Drama,missing,missing
1000199,3349,593,5,967749429,F,18,4,"Silence of the Lambs, The (1991)",Drama,Thriller,missing
1000200,5974,2028,4,956947181,F,25,1,Saving Private Ryan (1998),Action,Drama,War
1000201,721,3435,4,994084782,F,35,9,Double Indemnity (1944),Crime,Film-Noir,missing


In [4]:
# movies_dict = {}
# for i, row in data.iterrows():
#     movies_dict[row['item']] = row['name']

# movies_dict

In [5]:
# len(movies_dict)

In [6]:
data = data[['user', 'item', 'label', 'time']]
data_male = data_male[['user', 'item', 'label', 'time']]
data_female = data_female[['user', 'item', 'label', 'time']]
data

,user,item,label,time
0,5755,184,3,958280246
1,4585,519,3,964321944
2,1503,3114,4,974762175
3,2166,648,4,974614593
4,3201,2178,5,968626301
...,...,...,...,...
1000204,1509,2716,4,974747653
1000205,3756,1411,5,966096744
1000206,3609,1552,3,966614629
1000207,4169,3639,4,971579005


In [7]:
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
# DatasetPure is indicative of the fact we are not using any other feature other than the interaction between the user and an item.
#  Hence, this function builds the datasets from a Pure Collaborative Filtering perspective.

In [8]:
train_data_M, eval_data_M, test_data_M = random_split(data_male, multi_ratios=[0.8, 0.1, 0.1])
train_data_M, data_info_M= DatasetPure.build_trainset(train_data_M)
eval_data_M = DatasetPure.build_evalset(eval_data_M)
test_data_M = DatasetPure.build_testset(test_data_M)

train_data_F, eval_data_F, test_data_F = random_split(data_female, multi_ratios=[0.8, 0.1, 0.1])
train_data_F, data_info_F= DatasetPure.build_trainset(train_data_F)
eval_data_F = DatasetPure.build_evalset(eval_data_F)
test_data_F = DatasetPure.build_testset(test_data_F)

In [9]:
train_data
train_data_M

In [10]:
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=20,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

# monitor metrics on eval data during training
ncf.fit(
    train_data,
    neg_sampling=False, #for rating, this param is false else True
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)

# do final evaluation on test data
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)
#for implicit feedback, metrics like precision@k, recall@k, ndcg can be used

Training start time: 2023-12-06 20:15:09



2023-12-06 20:15:09,705 - WARNING - From C:\Users\14616\AppData\Roaming\Python\Python39\site-packages\keras\src\utils\version_utils.py:76: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.



Instructions for updating:
Colocations handled automatically by placer.


2023-12-06 20:15:09,730 - WARNING - From C:\Users\14616\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\normalization\batch_normalization.py:883: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


2023-12-06 20:15:09,939 - WARNING - From d:\anaconda\lib\site-packages\libreco\training\tf_trainer.py:120: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

train: 100%|██████████| 782/782 [00:03<00:00, 206.69it/s]


...

Epoch 20 elapsed: 3.092s
	 train_loss: 0.6129


eval_pointwise: 100%|██████████| 13/13 [00:00<00:00, 171.51it/s]


	 eval rmse: 0.9286


eval_pointwise: 100%|██████████| 13/13 [00:00<00:00, 177.18it/s]


{'loss': 0.9289411}

In [11]:
# predict preference of user 5755 to item 110
# ncf.predict(user=5755, item=110)

# # recommend 10items for user 5755
# ncf.recommend_user(user=5755, n_rec=10)

In [12]:
# ncfM = NCF(
#     task="rating",
#     data_info=data_info_M,
#     loss_type="cross_entropy",
#     embed_size=16,
#     n_epochs=20,
#     lr=1e-3,
#     batch_size=1024,
#     num_neg=1,
# )

# # monitor metrics on eval data during training
# ncf.fit(
#     train_data_M,
#     neg_sampling=False, #for rating, this param is false else True
#     verbose=2,
#     eval_data=eval_data_M,
#     metrics=["loss"],
# )

# # do final evaluation on test data
# evaluate(
#     model=ncf,
#     data=test_data_M,
#     neg_sampling=False,
#     metrics=["loss"],
# )

In [13]:
# ncfM = NCF(
#     task="rating",
#     data_info=data_info_M,
#     loss_type="cross_entropy",
#     embed_size=16,
#     n_epochs=20,
#     lr=1e-3,
#     batch_size=2048,
#     num_neg=1,
# )

# # monitor metrics on eval data during training
# ncf.fit(
#     train_data_M,
#     neg_sampling=False, #for rating, this param is false else True
#     verbose=2,
#     eval_data=eval_data_M,
#     metrics=["loss"],
# )

# # do final evaluation on test data
# evaluate(
#     model=ncf,
#     data=test_data_M,
#     neg_sampling=False,
#     metrics=["loss"],
# )

In [14]:
# ncf = NCF(
#     task="rating",
#     data_info=data_info_F,
#     loss_type="cross_entropy",
#     embed_size=16,
#     n_epochs=20,
#     lr=1e-3,
#     batch_size=2048,
#     num_neg=1,
# )

# # monitor metrics on eval data during training
# ncf.fit(
#     train_data_F,
#     neg_sampling=False, #for rating, this param is false else True
#     verbose=2,
#     eval_data=eval_data_F,
#     metrics=["loss"],
# )

# # do final evaluation on test data
# evaluate(
#     model=ncf,
#     data=test_data_F,
#     neg_sampling=False,
#     metrics=["loss"],
# )

In [15]:
# # predict preference of user 5755 to item 110
# ncf.predict(user=5755, item=110)

# # recommend 10items for user 5755
# ncf.recommend_user(user=5755, n_rec=10)